In [1]:
import sys, os
root = os.path.dirname(os.path.abspath(os.curdir))
if root not in sys.path:
    sys.path.insert(1, root)

import json

import importlib
import pandas as pd
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
from datetime import datetime
#from copy import copy
import csv

# import data_acquisition
from utils import sql_dictionary
from utils import data_acquisition
from validation import data_validation
from validation import data_validation_constants
os.chdir(root)

In [2]:
def function_validate_tfe_mep(  comparison_list
                              , path_to_compare_PRE
                              , path_to_compare_POST
                              , do_not_investigate
                              , validation_name
                              ):
    
    datasets_to_validate = []

    for comparison in comparison_list:
        path_PRE  = path_to_compare_PRE + comparison + '.csv'
        path_POST = path_to_compare_POST + comparison + '.csv'
        validation_dict = data_validation_constants.all_tfe_validation_constants[comparison]
        print('----------------------------------------------------------------')
        print(comparison)
        print('----------------------------------------------------------------')
        result_function_compare_dataset = data_validation.function_compare_dataset(validation_dict, path_PRE, path_POST)
        print(validation_dict)
        requires_validation = data_validation.function_compare_report(  path_to_compare_POST
                                                                      , comparison
                                                                      , result_function_compare_dataset
                                                                      , do_not_investigate = do_not_investigate
                                                                      , verbose = True
                                                                      )
        if requires_validation:
            datasets_to_validate.append(comparison)

    if len(datasets_to_validate)>0 and len(datasets_to_validate) < 1048576:
        df_to_validate = pd.DataFrame(datasets_to_validate, columns = ['dataset'])
        df_to_validate['status'] = ''
        path_for_report = path_to_compare_POST + validation_name + '.xlsx'
        print(path_for_report)
        df_to_validate.to_excel(path_for_report)
    else:
        df_to_validate = pd.DataFrame(datasets_to_validate, columns = ['dataset'])
        df_to_validate['status'] = ''
        path_for_report = path_to_compare_POST + validation_name + '.csv'
        print(path_for_report)
        df_to_validate.to_csv(path_for_report)
        
    return path_for_report

In [3]:
validation_name = 'test_mep_20230912_production_mode_2'
path_to_compare_PRE =  'C:/Prod/total-fund-exposure-MS/output/'+validation_name+'/'
path_to_compare_POST = 'C:/Development/total-fund-exposure-MS/output/'+validation_name+'/'
do_not_investigate = ['shared_duplicate','attributes_agg_analysis', 'data_agg_analysis']
comparison_list = ['dim_countries', 'dim_issuers', 'dim_gics', 'dim_portfolios', 'fact_portfolios', 'fact_all_constituents', 'fact_exposure']

In [4]:
dataset_to_compare = 'fact_exposure'
compare_dictionary = data_validation_constants.all_tfe_validation_constants[dataset_to_compare]
compare_dictionary['file_format'] = 'dataframe'

path_PRE  = path_to_compare_PRE  + dataset_to_compare + '.csv'
path_POST = path_to_compare_POST + dataset_to_compare + '.csv'

df_PRE  = pd.read_csv(path_PRE, delimiter = ';')
df_POST = pd.read_csv(path_POST, delimiter = ';')

df_PRE['Instrument_FinancialMaturityDate'] = df_PRE['Instrument_FinancialMaturityDate'].replace('NaT',float('nan'))
df_PRE['UltimateUnderlying_FinancialMaturityDate'] = df_PRE['UltimateUnderlying_FinancialMaturityDate'].replace('NaT',float('nan'))

compare_result = data_validation.function_compare_dataset(  compare_dictionary = compare_dictionary
                                                          , path_PRE = None
                                                          , path_POST = None
                                                          , df_PRE = df_PRE
                                                          , df_POST= df_POST
                                                          )

C:\Users\paratelle\AppData\Local\Temp\1\ipykernel_25120\3827952452.py:8: DtypeWarning: Columns (18,19,23,25,27,29,31,33,35,36,37,38,39,40,41,43,44,45,46,58,59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  df_PRE  = pd.read_csv(path_PRE, delimiter = ';')
C:\Users\paratelle\AppData\Local\Temp\1\ipykernel_25120\3827952452.py:9: DtypeWarning: Columns (18,19,23,25,27,29,31,33,35,36,37,38,39,40,41,43,44,45,46,54,58,60,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  df_POST = pd.read_csv(path_POST, delimiter = ';')


Int64
Issuer casting Instrument_GICSSectorCode to Int64. Reverting to original format
invalid literal for int() with base 10: 'N/D'
Issuer casting Instrument_GICSIndustryCode to Int64. Reverting to original format
invalid literal for int() with base 10: 'N/D'
float
date


In [6]:
for row in compare_result:
    print(row, compare_result[row]['result'])
    display(compare_result[row]['dataframe'])

duplicate_from_PRE 0


Empty DataFrame
Columns: [PositionDate, Object_Type, Portfolio_PSPPortfolioCode, Instrument_CurrencyCode, Instrument_ReportingCurrencyCode, Instrument_PSPInstrumentID, Leg_PSPInstrumentLegID, Instrument_Description, Instrument_RiskLocationCountryCode, Instrument_BICSBETAClassificationID, Instrument_Key]
Index: []

duplicate_from_POST 1118


PositionDate Object_Type Portfolio_PSPPortfolioCode  \
1006150   2023-06-30   Portfolio                     AB1282   
1006151   2023-06-30   Portfolio                     AB1282   
1006152   2023-06-30   Portfolio                     AB1282   
1006160   2023-06-30   Portfolio                     AB1282   
1006161   2023-06-30   Portfolio                     AB1282   
...              ...         ...                        ...   
1017386   2023-09-01   Portfolio                     AB1302   
1017387   2023-09-01   Portfolio                     AB1302   
1017388   2023-09-01   Portfolio                     AB1302   
1017437   2023-09-01   Portfolio                     FIF750   
1017438   2023-09-01   Portfolio                     FIF750   

        Instrument_CurrencyCode Instrument_ReportingCurrencyCode  \
1006150                     USD                              USD   
1006151                     USD                              USD   
1006152                     USD                              USD   
1006160                     EUR                              EUR   
1006161                     EUR                              EUR   
...                         ...                              ...   
1017386                     USD                              USD   
1017387                     USD                              USD   
1017388                     USD                              USD   
1017437                     USD                              USD   
1017438                     USD                              USD   

         Instrument_PSPInstrumentID  Leg_PSPInstrumentLegID  \
1006150                  -999999999                    <NA>   
1006151                  -999999999                    <NA>   
1006152                  -999999999                    <NA>   
1006160                  -999999999                    <NA>   
1006161                  -999999999                    <NA>   
...                             ...                     ...   
1017386                  -999999999                    <NA>   
1017387                  -999999999                    <NA>   
1017388                  -999999999                    <NA>   
1017437                  -999999999                    <NA>   
1017438                  -999999999                    <NA>   

                                    Instrument_Description  \
1006150  PFS20230331-AB1282M_BRENT CRUDE (ICE) DEC 24 -...   
1006151  PFS20230331-AB1282M_BRENT CRUDE (ICE) JUL 23 -...   
1006152  PFS20230331-AB1282M_BRENT CRUDE (ICE) JUN 23 -...   
1006160  PFS20230331-AB1282M_CDS: (ITRAXX.EUR.30.V2) - ...   
1006161  PFS20230331-AB1282M_CDS: (ITRAXX.EUR.39.V1) - ...   
...                                                    ...   
1017386  AB1302_923723[-1105722.26] LCOZ4 2024-10-28 50...   
1017387  AB1302_923723[-1105722.26] LCOZ4 2024-10-28 85...   
1017388  AB1302_923723[1105722.26] LCOZ4 2024-10-28 75....   
1017437    FIF750_CDS Index - CLN - replication (60% 5y)_1   
1017438    FIF750_CDS Index - Replication CLN  (40% 10y)_3   

        Instrument_RiskLocationCountryCode  \
1006150                                NaN   
1006151                                NaN   
1006152                                NaN   
1006160                                NaN   
1006161                                NaN   
...                                    ...   
1017386                                NaN   
1017387                                NaN   
1017388                                NaN   
1017437                                NaN   
1017438                                NaN   

         Instrument_BICSBETAClassificationID  Instrument_Key  
1006150                                 <NA>       113320226  
1006151                                 <NA>       113320227  
1006152                                 <NA>       113320228  
1006160                                 <NA>       113320236  
1006161                                 <NA>       113320237  
.

shared_duplicate 0


...

None

data_agg_analysis 7


data_name  total_0_0  total_0_1  total_1_0  total_1_1_D  \
0  Position_NetAssetValue_CAD          0          0          0            7   
0                 Exposure_FI          0          0          0            0   
0                 Exposure_CR          0          0          0            0   
0             Exposure_Issuer          0          0          0            7   
0             Exposure_Equity          0          0          0            0   
0                 Exposure_FX          0          0          0            7   
0          Exposure_Commodity          0          0          0          395   

   total_1_1_S  total_logic   total  
0       956095       956102  956102  
0       956102       956102  956102  
0       956102       956102  956102  
0       956095       956102  956102  
0       956102       956102  956102  
0       956095       956102  956102  
0       955707       956102  956102

comparison_data 416


data_name PositionDate Object_Type  \
999729  Position_NetAssetValue_CAD   2023-09-01   Portfolio   
999798  Position_NetAssetValue_CAD   2023-09-01   Portfolio   
999830  Position_NetAssetValue_CAD   2023-09-01   Portfolio   
999856  Position_NetAssetValue_CAD   2023-09-01   Portfolio   
999857  Position_NetAssetValue_CAD   2023-09-01   Portfolio   
...                            ...          ...         ...   
357235          Exposure_Commodity   2023-09-01   Portfolio   
357236          Exposure_Commodity   2023-09-01   Portfolio   
357237          Exposure_Commodity   2023-09-01   Portfolio   
357238          Exposure_Commodity   2023-09-01   Portfolio   
357239          Exposure_Commodity   2023-09-01   Portfolio   

       Portfolio_PSPPortfolioCode Instrument_CurrencyCode  \
999729                     PD1765                     USD   
999798                     PD1859                     USD   
999830                     PD1887                     USD   
999856                     PD1920                     USD   
999857                     PD1923                     EUR   
...                           ...                     ...   
357235                     AB1260                     USD   
357236                     AB1260                     MYR   
357237                     AB1260                     USD   
357238                     AB1260                     USD   
357239                     AB1260                     USD   

       Instrument_ReportingCurrencyCode  Instrument_PSPInstrumentID  \
999729                              USD                           0   
999798                              USD                           0   
999830                              USD                           0   
999856                              USD                           0   
999857                              EUR                           0   
...                                 ...                         ...   
357235                              USD                     2790016   
357236                              MYR                     2793793   
357237                              USD                     2807407   
357238                              USD                     2809641   
357239                              USD                     2811627   

        Leg_PSPInstrumentLegID   Instrument_Description  \
999729                       0           Not Applicable   
999798                       0           Not Applicable   
999830                       0           Not Applicable   
999856                       0           Not Applicable   
999857                       0           Not Applicable   
...                        ...                      ...   
357235                    <NA>  RED WHEAT FUT MGE Mar24   
357236                    <NA>  CRUDE PALM OIL FU Dec23   
357237                    <NA>  FCOJ-A FUTURE     Nov23   
357238                    <NA>  PALLADIUM FUTURE  Dec23   
357239                    <NA>  ROUGH RICE (CBOT) Nov23   

       Instrument_RiskLocationCountryCode  \
999729                                 DE   
999798                                 US   
999830                                 US   
999856                                 US   
999857                                 FR   
...                                   ...   
357235                                NaN   
357236                                NaN   
357237                                NaN   
357238                                NaN   
357239                                NaN   

        Instrument_BICSBETAClassificationID  Instrument_Key data_status  \
999729                                 6278            <NA>       1_1_D   
999798                                 6848            <NA>       1_1_D   
999830                                 5601            <NA>       1_1_D   
999856                                 6494            <NA>       1_1_D   
999857                                 2839            <N

In [4]:
print(function_validate_tfe_mep(  comparison_list
                                , path_to_compare_PRE
                                , path_to_compare_POST
                                , do_not_investigate
                                , validation_name)
)

----------------------------------------------------------------
dim_countries
----------------------------------------------------------------
{'file_format': 'csv', 'keys_columns': ['Country_ISOCode'], 'attributes_columns': ['Country_Name', 'Country_CurrencyCode', 'Country_IntermediateRegion', 'Country_SubRegion', 'Country_Region']}
duplicate_from_PRE 0
duplicate_from_POST 0
shared_duplicate 0
missing_from_PRE 0
missing_from_POST 0
attributes_agg_analysis 5


attribute_name  total_0_0  total_0_1  total_1_0  total_1_1_D  \
0                Country_Name          0          0          0            0   
0        Country_CurrencyCode          0          0          0            0   
0  Country_IntermediateRegion          1          0          0            0   
0           Country_SubRegion          1          0          0            0   
0              Country_Region          1          0          0            0   

   total_1_1_S  total_logic  total  
0          251          251    251  
0          251          251    251  
0          250          251    251  
0          250          251    251  
0          250          251    251

----------------------------------------------------------------
dim_issuers
----------------------------------------------------------------
Int64
float
{'file_format': 'csv', 'keys_columns': ['Issuer_Code'], 'attributes_columns': ['Issuer_Name', 'Issuer_PSPLegalEntityID', 'Issuer_UltimateParentCode', 'Issuer_RiskLocationCountryCode', 'Issuer_NormalizedCountryCode', 'Issuer_BICSBETAClassificationID', 'Issuer_BICSBETASectorCode', 'Issuer_BICSBETASectorName', 'Issuer_BICSBETAIndustryGroupCode', 'Issuer_BICSBETAIndustryGroupName', 'Issuer_BICSBETAIndustryCode', 'Issuer_BICSBETAIndustryName', 'Issuer_BICSBETASubIndustryCode', 'Issuer_BICSBETASubIndustryName', 'Issuer_BICSBETAActivityCode', 'Issuer_BICSBETAActivityName', 'Issuer_BICSBETASubActivityCode', 'Issuer_BICSBETASubActivityName', 'Issuer_BICSBETASegmentCode', 'Issuer_BICSBETASegmentName', 'Issuer_Name_UltimateParent', 'Issuer_UltimateParentPSPLegalEntityID', 'Issuer_UltimateParentRiskLocationCountryCode', 'Issuer_UltimateParentNorm

Issuer_Code                Issuer_Name_POST  \
24778       156523                         Sky Ltd   
24779      1459132  Desarrolladora Homex SAB de CV   
24780      7385813                Fiera Milano SpA   
24782     13538077                      Arcario AB   
24781     24106729        Akira Holding Foundation   
24786     27256337             Louis Dreyfus Co BV   
24785     28816089       Airborne Wireless Network   
24783     41860650    Home Partners Of America Inc   
24784     42058638                     Tricida Inc   

       Issuer_PSPLegalEntityID_POST  Issuer_UltimateParentCode_POST  \
24778                         27692                          101712   
24779                         12934                         1459132   
24780                         14733                         7385813   
24782                         63059                        13538077   
24781                         34990                        24106729   
24786                         20608                        24106729   
24785                         57893                        28816089   
24783                         54906                        41860650   
24784                         30204                        42058638   

      Issuer_RiskLocationCountryCode_POST Issuer_NormalizedCountryCode_POST  \
24778                                  GB                                GB   
24779                                  MX                                MX   
24780                                  IT                                IT   
24782                                  SE                                SE   
24781                                  LI                                LI   
24786                                  NL                                NL   
24785                                  US                                US   
24783                                  US                                US   
24784                                  US                                US   

       Issuer_BICSBETAClassificationID_POST  Issuer_BICSBETASectorCode_POST  \
24778                                  5363                              10   
24779                                  5544                              11   
24780                                  6469                              17   
24782                                  6042                              14   
24781                                  6025                              14   
24786                                  5920                              12   
24785                                  5459                              10   
24783                                  6063                              14   
24784                                  6200                              16   

      Issuer_BICSBETASectorName_POST  Issuer_BICSBETAIndustryGroupCode_POST  \
24778                 Communications                                   1010   
24779         Consumer Discretionary                                   1110   
24780                    Industrials                                   1711   
24782                     Financials                                   1411   
24781                     Financials                                   1411   
24786               Consumer Staples                                   1211   
24785                 Communications                                   1011   
24783                     Financials                                   1411   
24784                    Health Care                                   1610   

      Issuer_BICSBETAIndustryGroupName_POST  Issuer_BICSBETAIndustryCode_POST  \
24778                                 Media                            101011   
24779       Consumer Discretionary Products                            111012   
24780                   Industrial Services                            171111   
24782                    Financial Services               

missing_from_POST 0
attributes_agg_analysis 47


...

data_agg_analysis 7


data_name  total_0_0  total_0_1  total_1_0  total_1_1_D  \
0  Position_NetAssetValue_CAD          0          0          0            7   
0                 Exposure_FI          0          0          0            0   
0                 Exposure_CR          0          0          0            0   
0             Exposure_Issuer          0          0          0            7   
0             Exposure_Equity          0          0          0            0   
0                 Exposure_FX          0          0          0            7   
0          Exposure_Commodity          0          0          0          395   

   total_1_1_S  total_logic   total  
0       956095       956102  956102  
0       956102       956102  956102  
0       956102       956102  956102  
0       956095       956102  956102  
0       956102       956102  956102  
0       956095       956102  956102  
0       955707       956102  956102

comparison_data 416


data_name PositionDate Object_Type  \
999729  Position_NetAssetValue_CAD   2023-09-01   Portfolio   
999798  Position_NetAssetValue_CAD   2023-09-01   Portfolio   
999830  Position_NetAssetValue_CAD   2023-09-01   Portfolio   
999856  Position_NetAssetValue_CAD   2023-09-01   Portfolio   
999857  Position_NetAssetValue_CAD   2023-09-01   Portfolio   
...                            ...          ...         ...   
357235          Exposure_Commodity   2023-09-01   Portfolio   
357236          Exposure_Commodity   2023-09-01   Portfolio   
357237          Exposure_Commodity   2023-09-01   Portfolio   
357238          Exposure_Commodity   2023-09-01   Portfolio   
357239          Exposure_Commodity   2023-09-01   Portfolio   

       Portfolio_PSPPortfolioCode Instrument_CurrencyCode  \
999729                     PD1765                     USD   
999798                     PD1859                     USD   
999830                     PD1887                     USD   
999856                     PD1920                     USD   
999857                     PD1923                     EUR   
...                           ...                     ...   
357235                     AB1260                     USD   
357236                     AB1260                     MYR   
357237                     AB1260                     USD   
357238                     AB1260                     USD   
357239                     AB1260                     USD   

       Instrument_ReportingCurrencyCode  Instrument_PSPInstrumentID  \
999729                              USD                           0   
999798                              USD                           0   
999830                              USD                           0   
999856                              USD                           0   
999857                              EUR                           0   
...                                 ...                         ...   
357235                              USD                     2790016   
357236                              MYR                     2793793   
357237                              USD                     2807407   
357238                              USD                     2809641   
357239                              USD                     2811627   

        Leg_PSPInstrumentLegID   Instrument_Description  \
999729                       0           Not Applicable   
999798                       0           Not Applicable   
999830                       0           Not Applicable   
999856                       0           Not Applicable   
999857                       0           Not Applicable   
...                        ...                      ...   
357235                    <NA>  RED WHEAT FUT MGE Mar24   
357236                    <NA>  CRUDE PALM OIL FU Dec23   
357237                    <NA>  FCOJ-A FUTURE     Nov23   
357238                    <NA>  PALLADIUM FUTURE  Dec23   
357239                    <NA>  ROUGH RICE (CBOT) Nov23   

       Instrument_RiskLocationCountryCode  \
999729                                 DE   
999798                                 US   
999830                                 US   
999856                                 US   
999857                                 FR   
...                                   ...   
357235                                NaN   
357236                                NaN   
357237                                NaN   
357238                                NaN   
357239                                NaN   

        Instrument_BICSBETAClassificationID  Instrument_Key data_status  \
999729                                 6278            <NA>       1_1_D   
999798                                 6848            <NA>       1_1_D   
999830                                 5601            <NA>       1_1_D   
999856                                 6494            <NA>       1_1_D   
999857                                 2839            <N

C:/Development/total-fund-exposure-MS/output/test_mep_20230912_production_mode_2/test_mep_20230912_production_mode_2.xlsx
C:/Development/total-fund-exposure-MS/output/test_mep_20230912_production_mode_2/test_mep_20230912_production_mode_2.xlsx


In [11]:
display(df_PRE[df_PRE['Instrument_Description'] == 'EUR [USD] FX CROSS'])
display(df_POST[df_POST['Instrument_Description'] == 'EUR [USD] FX CROSS'])

PositionDate Object_Type                             Position_Type  \
1006707   2023-06-30   Portfolio  Pooled Fund/Private Membership with Data   
1006708   2023-06-30   Portfolio  Pooled Fund/Private Membership with Data   
1006709   2023-06-30   Portfolio  Pooled Fund/Private Membership with Data   
1006710   2023-06-30   Portfolio  Pooled Fund/Private Membership with Data   
1008749   2023-07-31   Portfolio  Pooled Fund/Private Membership with Data   
1008750   2023-07-31   Portfolio  Pooled Fund/Private Membership with Data   
1008751   2023-07-31   Portfolio  Pooled Fund/Private Membership with Data   
1008752   2023-07-31   Portfolio  Pooled Fund/Private Membership with Data   
1010782   2023-08-25   Portfolio  Pooled Fund/Private Membership with Data   
1010783   2023-08-25   Portfolio  Pooled Fund/Private Membership with Data   
1010784   2023-08-25   Portfolio  Pooled Fund/Private Membership with Data   
1010785   2023-08-25   Portfolio  Pooled Fund/Private Membership with Data   
1012817   2023-08-31   Portfolio  Pooled Fund/Private Membership with Data   
1012818   2023-08-31   Portfolio  Pooled Fund/Private Membership with Data   
1012819   2023-08-31   Portfolio  Pooled Fund/Private Membership with Data   
1012820   2023-08-31   Portfolio  Pooled Fund/Private Membership with Data   
1014686   2023-09-01   Portfolio  Pooled Fund/Private Membership with Data   
1014687   2023-09-01   Portfolio  Pooled Fund/Private Membership with Data   
1014688   2023-09-01   Portfolio  Pooled Fund/Private Membership with Data   
1014689   2023-09-01   Portfolio  Pooled Fund/Private Membership with Data   

        Position_Source Portfolio_PSPPortfolioCode  Portfolio_PSPPortfolioID  \
1006707           Ghost                     AB1287                     52243   
1006708           Ghost                     AB1287                     52243   
1006709           Ghost                     AB1287                     52243   
1006710           Ghost                     AB1287                     52243   
1008749           Ghost                     AB1287                     52243   
1008750           Ghost                     AB1287                     52243   
1008751           Ghost                     AB1287                     52243   
1008752           Ghost                     AB1287                     52243   
1010782           Ghost                     AB1287                     52243   
1010783           Ghost                     AB1287                     52243   
1010784           Ghost                     AB1287                     52243   
1010785           Ghost                     AB1287                     52243   
1012817           Ghost                     AB1287                     52243   
1012818           Ghost                     AB1287                     52243   
1012819           Ghost                     AB1287                     52243   
1012820           Ghost                     AB1287                     52243   
1014686           Ghost                     AB1287                     52243   
1014687           Ghost                     AB1287                     52243   
1014688           Ghost                     AB1287                     52243   
1014689           Ghost                     AB1287                     52243   

        Instrument_CurrencyCode Instrument_ReportingCurrencyCode  \
1006707                     EUR                              EUR   
1006708                     USD                              USD   
1006709                     EUR                              EUR   
1006710                     USD                              USD   
1008749                     EUR                              EUR   
1008750                     USD                              USD   
1008751                     EUR                              EUR   
1008752                     USD                              USD   
1010782                     EUR                              EUR   
1010783 

PositionDate Object_Type                             Position_Type  \
1006709   2023-06-30   Portfolio  Pooled Fund/Private Membership with Data   
1006710   2023-06-30   Portfolio  Pooled Fund/Private Membership with Data   
1006711   2023-06-30   Portfolio  Pooled Fund/Private Membership with Data   
1006712   2023-06-30   Portfolio  Pooled Fund/Private Membership with Data   
1008751   2023-07-31   Portfolio  Pooled Fund/Private Membership with Data   
1008752   2023-07-31   Portfolio  Pooled Fund/Private Membership with Data   
1008753   2023-07-31   Portfolio  Pooled Fund/Private Membership with Data   
1008754   2023-07-31   Portfolio  Pooled Fund/Private Membership with Data   
1010784   2023-08-25   Portfolio  Pooled Fund/Private Membership with Data   
1010785   2023-08-25   Portfolio  Pooled Fund/Private Membership with Data   
1010786   2023-08-25   Portfolio  Pooled Fund/Private Membership with Data   
1010787   2023-08-25   Portfolio  Pooled Fund/Private Membership with Data   
1012819   2023-08-31   Portfolio  Pooled Fund/Private Membership with Data   
1012820   2023-08-31   Portfolio  Pooled Fund/Private Membership with Data   
1012821   2023-08-31   Portfolio  Pooled Fund/Private Membership with Data   
1012822   2023-08-31   Portfolio  Pooled Fund/Private Membership with Data   
1014688   2023-09-01   Portfolio  Pooled Fund/Private Membership with Data   
1014689   2023-09-01   Portfolio  Pooled Fund/Private Membership with Data   
1014690   2023-09-01   Portfolio  Pooled Fund/Private Membership with Data   
1014691   2023-09-01   Portfolio  Pooled Fund/Private Membership with Data   

        Position_Source Portfolio_PSPPortfolioCode  Portfolio_PSPPortfolioID  \
1006709           Ghost                     AB1287                   52243.0   
1006710           Ghost                     AB1287                   52243.0   
1006711           Ghost                     AB1287                   52243.0   
1006712           Ghost                     AB1287                   52243.0   
1008751           Ghost                     AB1287                   52243.0   
1008752           Ghost                     AB1287                   52243.0   
1008753           Ghost                     AB1287                   52243.0   
1008754           Ghost                     AB1287                   52243.0   
1010784           Ghost                     AB1287                   52243.0   
1010785           Ghost                     AB1287                   52243.0   
1010786           Ghost                     AB1287                   52243.0   
1010787           Ghost                     AB1287                   52243.0   
1012819           Ghost                     AB1287                   52243.0   
1012820           Ghost                     AB1287                   52243.0   
1012821           Ghost                     AB1287                   52243.0   
1012822           Ghost                     AB1287                   52243.0   
1014688           Ghost                     AB1287                   52243.0   
1014689           Ghost                     AB1287                   52243.0   
1014690           Ghost                     AB1287                   52243.0   
1014691           Ghost                     AB1287                   52243.0   

        Instrument_CurrencyCode Instrument_ReportingCurrencyCode  \
1006709                     EUR                              EUR   
1006710                     USD                              USD   
1006711                     EUR                              EUR   
1006712                     USD                              USD   
1008751                     EUR                              EUR   
1008752                     USD                              USD   
1008753                     EUR                              EUR   
1008754                     USD                              USD   
1010784                     EUR                              EUR   
1010785 

### 8 - Agg Check on Exposure by PSPCategorization ID

In [7]:
key = ['PositionDate','Object_Type','Portfolio_PSPPortfolioCode','Instrument_PSPInstrumentCategorizationCode','Position_Type','Instrument_CurrencyCode','Instrument_ReportingCurrencyCode']
value = ['Position_NetAssetValue_CAD','Exposure_FI','Exposure_CR','Exposure_Issuer','Exposure_Equity','Exposure_FX','Exposure_Commodity']
agg_fact_exposure_dict = { 'file_format': 'dataframe'
                          , 'keys_columns': key
                          , 'data_columns': value
                          , 'data_format' : {  'float': value
                                             , 'date': ['PositionDate']
                                             }
                          , 'data_derror_tolerance' : 100000
                          , 'data_perror_tolerance' : 0.001
                         }
comparison = 'agg_exposure'
pre_agg_fact_exposure_df  = df_PRE[key + value].groupby(key, dropna = False).sum().reset_index()
post_agg_fact_exposure_df = df_POST[key + value].groupby(key, dropna = False).sum().reset_index()

result_function_compare_dataset = data_validation.function_compare_dataset(agg_fact_exposure_dict, df_PRE = pre_agg_fact_exposure_df, df_POST = post_agg_fact_exposure_df)
print(data_validation.function_compare_report(  path_to_compare_POST
                                              , comparison
                                              , result_function_compare_dataset
                                              , do_not_investigate = do_not_investigate
                                              , verbose = True
                                              ))

float
date
duplicate_from_PRE 0
duplicate_from_POST 0
shared_duplicate 0
missing_from_PRE 221


PositionDate Object_Type Portfolio_PSPPortfolioCode  \
33550   2023-06-30   Benchmark                     SA0598   
33551   2023-06-30   Benchmark                     SA0856   
33552   2023-06-30   Portfolio                     AB1224   
33553   2023-06-30   Portfolio                     AB1260   
33554   2023-06-30   Portfolio                     AB1260   
...            ...         ...                        ...   
33766   2023-09-01   Portfolio                     CEA527   
33767   2023-09-01   Portfolio                     CEA527   
33768   2023-09-01   Portfolio                     CEA527   
33769   2023-09-01   Portfolio                     CEA527   
33770   2023-09-01   Portfolio                     FEF643   

      Instrument_PSPInstrumentCategorizationCode  \
33550                         Fixed Income Index   
33551                         Fixed Income Index   
33552                               Common Share   
33553                       Equity Index Futures   
33554                       Equity Index Futures   
...                                          ...   
33766          Contract For Difference on Equity   
33767          Contract For Difference on Equity   
33768          Contract For Difference on Equity   
33769          Contract For Difference on Equity   
33770                 Warrants - Exchange Traded   

                                  Position_Type Instrument_CurrencyCode  \
33550                                  Internal                     CAD   
33551                                  Internal                     CAD   
33552  Pooled Fund/Private Membership with Data                     CHF   
33553                                  Internal                     USD   
33554                                  Internal                     USD   
...                                         ...                     ...   
33766  Pooled Fund/Private Membership with Data                     TRY   
33767  Pooled Fund/Private Membership with Data                     TWD   
33768  Pooled Fund/Private Membership with Data                     USD   
33769  Pooled Fund/Private Membership with Data                     ZAR   
33770                                  Internal                     USD   

      Instrument_ReportingCurrencyCode  Position_NetAssetValue_CAD_POST  \
33550                              CAD                            -0.00   
33551                              CAD                            -0.00   
33552                              ZAR                       1812255.57   
33553                              CNY                             0.00   
33554                              TWD                             0.00   
...                                ...                              ...   
33766                              N/D                             0.00   
33767                              N/D                             0.00   
33768                              N/D                             0.00   
33769                              N/D                             0.00   
33770                              CNY                             0.00   

       Exposure_FI_POST  Exposure_CR_POST  Exposure_Issuer_POST  \
33550               0.0               0.0                 -0.00   
33551               0.0               0.0                 -0.00   
33552               0.0               0.0                  0.00   
33553               0.0               0.0           -4752866.55   
33554               0.0               0.0            1529147.70   
...                 ...               ...                   ...   
33766               0.0               0.0             907840.00   
33767               0.0               0.0            4388085.00   
33768               0.0               0.0           20787693.00   
33769               0.0               0.0            4506005.00   
33770               0.0               0.0            4271367.91   

       Exposure_Equity_POST  Exposure_FX_

missing_from_POST 11


PositionDate Object_Type Portfolio_PSPPortfolioCode  \
2828    2023-06-30   Portfolio                     AB1307   
2966    2023-06-30   Portfolio                     CEA527   
9472    2023-07-31   Portfolio                     AB1307   
9611    2023-07-31   Portfolio                     CEA527   
16084   2023-08-25   Portfolio                     AB1307   
16231   2023-08-25   Portfolio                     CEA527   
22724   2023-08-31   Portfolio                     AB1307   
22870   2023-08-31   Portfolio                     CEA527   
29511   2023-09-01   Portfolio                     AB1307   
29658   2023-09-01   Portfolio                     CEA527   
33507   2023-09-01   Portfolio                     SA1254   

      Instrument_PSPInstrumentCategorizationCode  \
2828                     Credit Default Swaption   
2966           Contract For Difference on Equity   
9472                     Credit Default Swaption   
9611           Contract For Difference on Equity   
16084                    Credit Default Swaption   
16231          Contract For Difference on Equity   
22724                    Credit Default Swaption   
22870          Contract For Difference on Equity   
29511                    Credit Default Swaption   
29658          Contract For Difference on Equity   
33507                                       Cash   

                                  Position_Type Instrument_CurrencyCode  \
2828   Pooled Fund/Private Membership with Data                     N/D   
2966   Pooled Fund/Private Membership with Data                     N/D   
9472   Pooled Fund/Private Membership with Data                     N/D   
9611   Pooled Fund/Private Membership with Data                     N/D   
16084  Pooled Fund/Private Membership with Data                     N/D   
16231  Pooled Fund/Private Membership with Data                     N/D   
22724  Pooled Fund/Private Membership with Data                     N/D   
22870  Pooled Fund/Private Membership with Data                     N/D   
29511  Pooled Fund/Private Membership with Data                     N/D   
29658  Pooled Fund/Private Membership with Data                     N/D   
33507                                  Internal                     USD   

      Instrument_ReportingCurrencyCode  Position_NetAssetValue_CAD_PRE  \
2828                               N/D                            0.00   
2966                               N/D                            0.00   
9472                               N/D                            0.00   
9611                               N/D                            0.00   
16084                              N/D                            0.00   
16231                              N/D                            0.00   
22724                              N/D                            0.00   
22870                              N/D                            0.00   
29511                              N/D                            0.00   
29658                              N/D                            0.00   
33507                              USD                           86.28   

       Exposure_FI_PRE  Exposure_CR_PRE  Exposure_Issuer_PRE  \
2828               0.0              0.0                 94.4   
2966               0.0              0.0           31130254.0   
9472               0.0              0.0                 94.4   
9611               0.0              0.0           31084978.0   
16084              0.0              0.0                 94.4   
16231              0.0              0.0           36083371.0   
22724              0.0              0.0                 94.4   
22870              0.0              0.0           33835543.0   
29511              0.0              0.0                 94.4   
29658              0.0              0.0           38739574.0   
33507              0.0              0.0                  0.0   

       Exposure_Equity_PRE  Exposure_FX_PRE  Exposure_Commodity_PRE  
2828               

data_agg_analysis 7


data_name  total_0_0  total_0_1  total_1_0  total_1_1_D  \
0  Position_NetAssetValue_CAD          0          0          0         6456   
0                 Exposure_FI          0          0          0          204   
0                 Exposure_CR          0          0          0            0   
0             Exposure_Issuer          0          0          0          915   
0             Exposure_Equity          0          0          0          821   
0                 Exposure_FX          0          0          0         1144   
0          Exposure_Commodity          0          0          0           50   

   total_1_1_S  total_logic  total  
0        27083        33539  33539  
0        33335        33539  33539  
0        33539        33539  33539  
0        32624        33539  33539  
0        32718        33539  33539  
0        32395        33539  33539  
0        33489        33539  33539

comparison_data 9590


data_name PositionDate Object_Type  \
49     Position_NetAssetValue_CAD   2023-06-30   Benchmark   
51     Position_NetAssetValue_CAD   2023-06-30   Benchmark   
77     Position_NetAssetValue_CAD   2023-06-30   Benchmark   
89     Position_NetAssetValue_CAD   2023-06-30   Benchmark   
92     Position_NetAssetValue_CAD   2023-06-30   Benchmark   
...                           ...          ...         ...   
28446          Exposure_Commodity   2023-09-01   Portfolio   
28447          Exposure_Commodity   2023-09-01   Portfolio   
28448          Exposure_Commodity   2023-09-01   Portfolio   
28453          Exposure_Commodity   2023-09-01   Portfolio   
29278          Exposure_Commodity   2023-09-01   Portfolio   

      Portfolio_PSPPortfolioCode Instrument_PSPInstrumentCategorizationCode  \
49                        CEA513                               Common Share   
51                        CEA527                               Common Share   
77                        FEA613                               Common Share   
89                        FED629                               Common Share   
92                        FED629                               Common Share   
...                          ...                                        ...   
28446                     AB1282                             Not Determined   
28447                     AB1282                             Not Determined   
28448                     AB1282                             Not Determined   
28453                     AB1282                             Not Determined   
29278                     AB1302                             Not Determined   

                                  Position_Type Instrument_CurrencyCode  \
49                                     Internal                     CAD   
51                                     Internal                     CAD   
77                                     Internal                     USD   
89                                     Internal                     AUD   
92                                     Internal                     EUR   
...                                         ...                     ...   
28446  Pooled Fund/Private Membership with Data                     CNY   
28447  Pooled Fund/Private Membership with Data                     EUR   
28448  Pooled Fund/Private Membership with Data                     GBP   
28453  Pooled Fund/Private Membership with Data                     USD   
29278  Pooled Fund/Private Membership with Data                     USD   

      Instrument_ReportingCurrencyCode data_status  data_value_pre  \
49                                 CAD       1_1_D   -6.011785e+09   
51                                 CAD       1_1_D   -1.357041e+09   
77                                 USD       1_1_D   -5.488605e+09   
89                                 AUD       1_1_D   -6.959676e+07   
92                                 EUR       1_1_D   -3.355263e+08   
...                                ...         ...             ...   
28446                              CNY       1_1_D   -6.131080e+05   
28447                              EUR       1_1_D    0.000000e+00   
28448                              GBP       1_1_D    0.000000e+00   
28453                              USD       1_1_D   -6.131080e+05   
29278                              USD       1_1_D    9.721118e+07   

       data_value_post   data_derror  data_perror  
49       -6.206013e+09 -1.942286e+08     0.032308  
51       -1.400885e+09 -4.384325e+07     0.032308  
77       -5.495177e+09 -6.572046e+06     0.001197  
89       -7.033886e+07 -7.420978e+05     0.010663  
92       -3.367415e+08 -1.215172e+06     0.003622  
...                ...           ...          ...  
28446    -3.350086e+06 -2.736978e+06     4.464104  
28447    -1.469560e+07 -1.469560e+07         -inf  
28448    -4.480628e+06 -4.480628e+06         -inf  
28453    -5.396320e+05  7.347600e+04    -0.119842  
29278     1.9442

True


In [ ]:
key = ['PositionDate','Object_Type','Portfolio_PSPPortfolioCode','Instrument_PSPInstrumentCategorizationCode']
value = ['Position_NetAssetValue_CAD','Exposure_FI','Exposure_CR','Exposure_Issuer','Exposure_Equity','Exposure_FX','Exposure_Commodity']
agg_fact_exposure_dict = { 'file_format': 'dataframe'
                          , 'keys_columns': key
                          , 'data_columns': value
                          , 'data_format' : {  'float': value
                                             , 'date': ['PositionDate']
                                             }
                          , 'data_derror_tolerance' : 100000
                          , 'data_perror_tolerance' : 0.001
                         }
comparison = 'agg_exposure'
pre_agg_fact_exposure_df  = pre_fact_exposure_df[key + value].groupby(key, dropna = False).sum().reset_index()
post_agg_fact_exposure_df = post_fact_exposure_df[key + value].groupby(key, dropna = False).sum().reset_index()

result_function_compare_dataset = data_validation.function_compare_dataset(agg_fact_exposure_dict, df_PRE = pre_agg_fact_exposure_df, df_POST = post_agg_fact_exposure_df)
print(data_validation.function_compare_report(  path_to_compare_POST
                                              , comparison
                                              , result_function_compare_dataset
                                              , do_not_investigate = do_not_investigate
                                              , verbose = True
                                              ))

In [ ]:
filter_test_pre  = pre_fact_exposure_df['Instrument_PSPInstrumentID'] == 2656250
filter_test_post = post_fact_exposure_df['Instrument_PSPInstrumentID'] == 2656250

display(post_fact_exposure_df[filter_test_post])
display(pre_fact_exposure_df[filter_test_pre])

PositionDate Object_Type                             Position_Type  \
650104   2023-06-30   Portfolio  Pooled Fund/Private Membership with Data   
755239   2023-07-31   Portfolio  Pooled Fund/Private Membership with Data   
861328   2023-08-04   Portfolio  Pooled Fund/Private Membership with Data   
969482   2023-08-11   Portfolio  Pooled Fund/Private Membership with Data   

                        Position_Source Portfolio_PSPPortfolioCode  \
650104  Market Risk Analytics Positions                     AB1307   
755239  Market Risk Analytics Positions                     AB1307   
861328  Market Risk Analytics Positions                     AB1307   
969482  Market Risk Analytics Positions                     AB1307   

        Portfolio_PSPPortfolioID Instrument_CurrencyCode  \
650104                     57468                     EUR   
755239                     57468                     EUR   
861328                     57468                     EUR   
969482                     57468                     EUR   

       Instrument_ReportingCurrencyCode  Instrument_PSPInstrumentID  \
650104                              N/D                     2656250   
755239                              N/D                     2656250   
861328                              N/D                     2656250   
969482                              N/D                     2656250   

        Instrument_PSPInstrumentCategorizationID  \
650104                                      1152   
755239                                      1152   
861328                                      1152   
969482                                      1152   

       Instrument_PSPInstrumentCategorizationCode    Instrument_Description  \
650104                    Credit Default Swaption  ITRXXO38 05/17/23 PAY600   
755239                    Credit Default Swaption  ITRXXO38 05/17/23 PAY600   
861328                    Credit Default Swaption  ITRXXO38 05/17/23 PAY600   
969482                    Credit Default Swaption  ITRXXO38 05/17/23 PAY600   

        UltimateUnderlying_PSPInstrumentID  \
650104                             2656250   
755239                             2656250   
861328                             2656250   
969482                             2656250   

        UltimateUnderlying_PSPInstrumentCategorizationID  \
650104                                              1152   
755239                                              1152   
861328                                              1152   
969482                                              1152   

       UltimateUnderlying_PSPInstrumentCategorizationCode  \
650104                            Credit Default Swaption   
755239                            Credit Default Swaption   
861328                            Credit Default Swaption   
969482                            Credit Default Swaption   

       UltimateUnderlying_Description  UltimateUnderlying_IssuerCode  \
650104       ITRXXO38 05/17/23 PAY600                            0.0   
755239       ITRXXO38 05/17/23 PAY600                            0.0   
861328       ITRXXO38 05/17/23 PAY600                            0.0   
969482       ITRXXO38 05/17/23 PAY600                            0.0   

        UltimateUnderlying_IndexProxyPSPInstrumentID  \
650104                                       2656250   
755239                                       2656250   
861328                                       2656250   
969482                                       2656250   

       Instrument_RiskLocationCountryCode Instrument_NormalizedCountryCode  \
650104                                NaN                              NaN   
755239                                NaN                              NaN   
861328                                NaN                              NaN   
969482                                NaN                              NaN   

        Instrument_GeographyName  Instrument_BICSBETAClassificationID  \
650104      

PositionDate Object_Type                             Position_Type  \
650104   2023-06-30   Portfolio  Pooled Fund/Private Membership with Data   
755239   2023-07-31   Portfolio  Pooled Fund/Private Membership with Data   
861328   2023-08-04   Portfolio  Pooled Fund/Private Membership with Data   
969482   2023-08-11   Portfolio  Pooled Fund/Private Membership with Data   

                        Position_Source Portfolio_PSPPortfolioCode  \
650104  Market Risk Analytics Positions                     AB1307   
755239  Market Risk Analytics Positions                     AB1307   
861328  Market Risk Analytics Positions                     AB1307   
969482  Market Risk Analytics Positions                     AB1307   

        Portfolio_PSPPortfolioID Instrument_CurrencyCode  \
650104                     57468                     EUR   
755239                     57468                     EUR   
861328                     57468                     EUR   
969482                     57468                     EUR   

       Instrument_ReportingCurrencyCode  Instrument_PSPInstrumentID  \
650104                              N/D                     2656250   
755239                              N/D                     2656250   
861328                              N/D                     2656250   
969482                              N/D                     2656250   

        Instrument_PSPInstrumentCategorizationID  \
650104                                      1152   
755239                                      1152   
861328                                      1152   
969482                                      1152   

       Instrument_PSPInstrumentCategorizationCode    Instrument_Description  \
650104                    Credit Default Swaption  ITRXXO38 05/17/23 PAY600   
755239                    Credit Default Swaption  ITRXXO38 05/17/23 PAY600   
861328                    Credit Default Swaption  ITRXXO38 05/17/23 PAY600   
969482                    Credit Default Swaption  ITRXXO38 05/17/23 PAY600   

        UltimateUnderlying_PSPInstrumentID  \
650104                             2656250   
755239                             2656250   
861328                             2656250   
969482                             2656250   

        UltimateUnderlying_PSPInstrumentCategorizationID  \
650104                                              1152   
755239                                              1152   
861328                                              1152   
969482                                              1152   

       UltimateUnderlying_PSPInstrumentCategorizationCode  \
650104                            Credit Default Swaption   
755239                            Credit Default Swaption   
861328                            Credit Default Swaption   
969482                            Credit Default Swaption   

       UltimateUnderlying_Description  UltimateUnderlying_IssuerCode  \
650104       ITRXXO38 05/17/23 PAY600                            0.0   
755239       ITRXXO38 05/17/23 PAY600                            0.0   
861328       ITRXXO38 05/17/23 PAY600                            0.0   
969482       ITRXXO38 05/17/23 PAY600                            0.0   

        UltimateUnderlying_IndexProxyPSPInstrumentID  \
650104                                       2656250   
755239                                       2656250   
861328                                       2656250   
969482                                       2656250   

       Instrument_RiskLocationCountryCode Instrument_NormalizedCountryCode  \
650104                                NaN                              NaN   
755239                                NaN                              NaN   
861328                                NaN                              NaN   
969482                                NaN                              NaN   

        Instrument_GeographyName  Instrument_BICSBETAClassificationID  \
650104      

In [3]:
#os.chdir(root)
#
#date_dict = {  '2022-04': ['2022-04-01', '2022-04-08', '2022-04-15', '2022-04-22', '2022-04-29']
#             , '2022-05': ['2022-05-06', '2022-05-13', '2022-05-20', '2022-05-27', '2022-05-31']
#             , '2022-06': ['2022-06-03', '2022-06-10', '2022-06-17', '2022-06-24', '2022-06-30']
#             , '2022-07': ['2022-07-01', '2022-07-08', '2022-07-15', '2022-07-22', '2022-07-29']
#             , '2022-08': ['2022-08-05', '2022-08-12', '2022-08-19', '2022-08-26', '2022-08-31']
#             , '2022-09': ['2022-09-02', '2022-09-09', '2022-09-16', '2022-09-23', '2022-09-30']
#             , '2022-10': ['2022-10-07', '2022-10-14', '2022-10-21', '2022-10-28', '2022-10-31']
#             , '2022-11': ['2022-11-04', '2022-11-11', '2022-11-18', '2022-11-25', '2022-11-30']
#             , '2022-12': ['2022-12-02', '2022-12-09', '2022-12-16', '2022-12-23', '2022-12-30']
#             , '2023-01': ['2023-01-06', '2023-01-13', '2023-01-20', '2023-01-27', '2023-01-31']
#             , '2023-02': ['2023-02-03', '2023-02-10', '2023-02-17', '2023-02-24', '2023-02-28']
#             , '2023-03': ['2023-03-03', '2023-03-10', '2023-03-17', '2023-03-24', '2023-03-31']
#             , '2023-04': ['2023-04-07', '2023-04-14', '2023-04-21', '2023-04-28']
#             , '2023-05': ['2023-05-05', '2023-05-12', '2023-05-19', '2023-05-31']
#             , '2023-06': ['2023-06-02', '2023-06-09', '2023-06-16', '2023-06-23', '2023-06-30']
#             , '2023-07': ['2023-07-07', '2023-07-14']
#            }
#
#for year_month in date_dict:
#
#    current_dates = date_dict[year_month]
#    print(current_dates)
#    pam_calculator = PAM(
#        evaluation_date = "2023-05-23",
#        position_dates = current_dates,
#        psp_portfolio_ids = None,
#        exclusions_psp_instrument_ids = ["138514"],
#        force_refresh = False,
#        country_to_use = None,
#        sql_dictionary = sql_dictionary.sql_dictionary,
#        initialized_adaconnect = initialized_adaconnect,
#        file_path = "output/"
#    )
#
#    pam_calculator.generate_csv_files()

TypeError: unhashable type: 'list'